In [76]:
# import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
data=pd.read_csv('Dataset .csv')  # load the dataset
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [77]:
data.isnull().sum()
# replacing null values
data['Cuisines'].fillna(data['Cuisines'].mode())
# preprocess the dataset
data['Combined_features']=data['Cuisines'].fillna("")+' '+data['Price range'].astype(str)
# implement TF-IDF Vectorizer
tfidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(data['Combined_features'])
# calculate cosine similarity matrix
cosine_similarity=linear_kernel(tfidf_matrix, tfidf_matrix)
print(data.head())

   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 

In [125]:
# function to get recommendations
def recommend_restaurants(location, cosine_similarity=cosine_similarity):
    if location not in data['City'].values:
      return "Not Found"
    idx=data.index[data['City']==location].tolist()[0]
    sim_scores=list(enumerate(cosine_similarity[idx]))
    sim_scores=sorted(sim_scores, key=lambda x: x[1])
    sim_scores=sim_scores[1:10]
    restaurant_indices=[i[0] for i in sim_scores]
    return data[['Restaurant Name', 'Cuisines', 'Price range', 'City', 'Aggregate rating']].iloc[restaurant_indices]

In [131]:
# test the recommendation system
RR=recommend_restaurants('Taguig City')
print("Relevant Restaurants in Recommendations:")
print(RR)

Relevant Restaurants in Recommendations:
                             Restaurant Name  \
2                     Heat - Edsa Shangri-La   
3                                       Ooma   
5                               Din Tai Fung   
6                                 Buffet 101   
7                                    Vikings   
8   Spiral - Sofitel Philippine Plaza Manila   
9                                   Locavore   
10                          Silantro Fil-Mex   
12                          Silantro Fil-Mex   

                              Cuisines  Price range              City  \
2     Seafood, Asian, Filipino, Indian            4  Mandaluyong City   
3                      Japanese, Sushi            4  Mandaluyong City   
5                              Chinese            3  Mandaluyong City   
6                      Asian, European            4        Pasay City   
7   Seafood, Filipino, Asian, European            4        Pasay City   
8              European, Asian, Indian  

# Conclusion:
# 1-Relevant features(cuisines, price range, locality) are extracted.
# 2-Cosine similarity is used to measure the similarity between restaurants based on their features.
# 3-Given a user's preferred location, the system finds the most similar restaurants and recommends them.